Aplicamos lo aprendido en las dos actividades en las que hicimos uso del dataset de DESI.
Es decir, lo que haremos es:
1.  Aplicar GridSearchCV para encontrar los mejores hiperparámetros de la red convolucional que elaboré como tarea y que clasifica espectros astronómicos (GALAXY, STAR, QSO).
2.  Encontrar los mejores hiperparámetros para predecir el redshif de los espectros.

1. Clasificacion espectral.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras import models, layers

# Cargamos los datos
df_spectra = pd.read_csv(r'C:\Users\op354\OneDrive\Documentos\Oscar\UG\Repositorio Github\ejerciosytareas-Oscarraul28\balanced_spectra_features.csv')

# Cargamos el archivo de espectros
datos = np.load(r'C:\Users\op354\OneDrive\Documentos\Oscar\UG\Repositorio Github\ejerciosytareas-Oscarraul28\balanced_spectra_flux.npz', allow_pickle=True)
targetids = datos['targetids']
f_arrays = datos['flux_arrays']

# Preparamos los datos
X = []
Y = []

for i in range(len(targetids)):
    spectype_reng = df_spectra.loc[df_spectra['targetid'] == targetids[i], 'spectype']
    if spectype_reng.empty:
        continue
    spectype = spectype_reng.values[0]

    if spectype == 'GALAXY':
        Y.append(0)
    elif spectype == 'STAR':
        Y.append(1)
    elif spectype == 'QSO':
        Y.append(2)
    else:
        continue
    X.append(f_arrays[i])

X = np.asarray(X)
Y = np.asarray(Y)

print("Formas finales:")
print("X:", X.shape)
print("Y:", Y.shape)

Formas finales:
X: (492, 7781)
Y: (492,)


In [2]:
# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=112)

# Escalamiento de datos
escalador = StandardScaler()
X_train_esc = escalador.fit_transform(X_train)
X_test_esc = escalador.transform(X_test)

# Expandimos dimensión para Conv1D
X_train_esc = np.expand_dims(X_train_esc, axis=-1)
X_test_esc = np.expand_dims(X_test_esc, axis=-1)

print("Shape de entrenamiento:", X_train_esc.shape)

# Guardamos la forma de entrada para usarla en las funciones
INPUT_SHAPE = (X_train_esc.shape[1], 1)

Shape de entrenamiento: (344, 7781, 1)
